In [3]:
import pandas as pd

In [4]:
data=pd.read_csv(r"C:\Users\Varun\Music\Python\playground-series-s3e8\train.csv")

In [ ]:
data.info()
data.drop(labels=["id"],axis=1,inplace=True)
data

In [ ]:
data.duplicated().sum()

In [7]:
cat_columns=data.columns[data.dtypes=="object"]
num_columns=data.columns[data.dtypes!="object"]

In [ ]:
data[num_columns].describe().T

In [ ]:
for col in cat_columns:
    print(data[col].value_counts())
    print("=================")

In [11]:
# Ordianal Encoding
cut_map={"Fair":1,"Good":2,"Very Good":3,"Premium":4,"Ideal":5}
clarity_map={"I1":1,"SI2":2,"SI3":3,"VS2":4,"VS1":5,"VVS2":6,"VVS1":7,"IF":8}
color_map={"D":1,"E":2,"F":3,"G":4,"H":5,"I":6,"J":7}

In [12]:
data["cut"]=data["cut"].map(cut_map)
data["clarity"]=data["clarity"].map(clarity_map)
data["color"]=data["color"].map(color_map)